In [44]:
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('..\sell-in.txt',sep='\t')

In [46]:
prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')

In [47]:
df_grouped = df.groupby(['periodo', 'product_id']).sum().reset_index()

In [48]:
df_grouped = df_grouped[['product_id', 'periodo', 'tn']]
df_grouped.sort_values(by=['product_id', 'periodo'], inplace=True)
df_grouped.reset_index(drop=True, inplace=True)

In [49]:
df_grouped.head()

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


In [50]:
prod_stats = df_grouped.groupby('product_id').apply(lambda group: pd.Series({
    'primer_periodo': group['periodo'].min(),
    'ultimo_periodo': group['periodo'].max(),
    'values': group['tn'].count(),
    'values': group['tn'].count(),
    'total_tn': group['tn'].sum(),
    'average_tn': group['tn'].mean(),
    'median_tn': group['tn'].median(),
    'std_dev_tn': group['tn'].std(),
    'iqr_tn': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25))
}))
prod_stats['primer_periodo'] = prod_stats['primer_periodo'].astype(int)
prod_stats['primer_periodo'] = pd.to_datetime(prod_stats['primer_periodo'].astype(str), format='%Y%m')

prod_stats['ultimo_periodo'] = prod_stats['ultimo_periodo'].astype(int)
prod_stats['ultimo_periodo'] = pd.to_datetime(prod_stats['ultimo_periodo'].astype(str), format='%Y%m')

prod_stats.reset_index(inplace=True)

In [51]:
prod_stats = prod_stats[prod_stats['product_id'].isin(prod_a_predecir['product_id'])]
df_grouped = df_grouped[df_grouped['product_id'].isin(prod_a_predecir['product_id'])]

In [52]:
prod_stats.head()

,product_id,primer_periodo,ultimo_periodo,values,total_tn,average_tn,median_tn,std_dev_tn,iqr_tn
0,20001,2017-01-01,2019-12-01,36.0,50340.39558,1398.344322,1418.023430,298.145460,335.515348
1,20002,2017-01-01,2019-12-01,36.0,36337.25439,1009.368178,992.005505,303.834835,246.766150
2,20003,2017-01-01,2019-12-01,36.0,32004.15274,889.004243,786.715735,292.036581,260.440887
3,20004,2017-01-01,2019-12-01,36.0,24178.15379,671.615383,596.909300,224.450085,279.662265
4,20005,2017-01-01,2019-12-01,36.0,23191.21852,644.200514,578.571990,218.273222,305.415543


In [53]:
periodos_manual = [202001, 202002]

for per in periodos_manual:
    prod_a_pred = prod_a_predecir
    prod_a_pred['periodo'] = per
    prod_a_pred['tn'] = np.NaN
    df_grouped = pd.concat([df_grouped, prod_a_pred], axis=0)

df_grouped.sort_values(by=['product_id', 'periodo'], inplace=True)
df_grouped.reset_index(drop=True, inplace=True)

In [54]:
df_grouped

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132
...,...,...,...
23904,21276,201910,0.02079
23905,21276,201911,0.03341
23906,21276,201912,0.00892
23907,21276,202001,NaN


In [55]:
df_grouped.to_parquet('sell_in_group.parquet', index=False)
prod_stats.to_parquet('prod_stats.parquet', index=False)